In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

In [2]:
filepath_home= 'D:/Dacon_Psychological_disposition/'
filepath_out = 'C:/임시/Dacon_Psychological_disposition/' 
train = pd.read_csv(filepath_home+'train.csv',index_col=0)
test = pd.read_csv(filepath_home+'test_x.csv',index_col=0)

In [3]:
# 분할 지점 저장해두고 데이터 통합 후 전처리
split_point = train.shape[0]
data = pd.concat([train.drop('voted',axis=1),test],ignore_index=True)

In [4]:
data.shape

(56915, 76)

# NA handle

### __Familysize__

In [5]:
data.familysize[(data.familysize<0)|(data.familysize.isnull())] # 결측 없음

Series([], Name: familysize, dtype: int64)

In [6]:
data.familysize.sort_values(ascending=False)[:10] # 눈에 띄는 이상치 3개 (2147483647 , 999 , 100)

24598    2147483647
379             999
25661           100
21567            44
34847            44
12056            44
28111            34
41326            30
48605            23
34749            21
Name: familysize, dtype: int64

train 데이터에 포함된 데이터임을 확인 후 이상치 제거

In [7]:
outlier_idx = data.familysize[data.familysize>99].index
for idx in outlier_idx:
    if idx < split_point:
        data = data.drop(idx,axis=0)
        split_point-=1
data.shape # 3개 행 모두 제거

(56912, 76)

### __Education__

In [8]:
data.education.unique() # NA 는 없지만 0 (무응답 존재)

array([2, 4, 3, 1, 0], dtype=int64)

In [9]:
data.education.value_counts() # 무응답 데이터 650개

2    21068
3    19025
4     9817
1     6352
0      650
Name: education, dtype: int64

In [10]:
# 스피어만 상관계수 기준 married 칼럼과 약한 상관관계를 보임
data.corr(method='spearman')['education'].abs().sort_values(ascending=False) 

education    1.000000
married      0.278788
wr_03        0.220288
wr_06        0.196406
QbA          0.153425
               ...   
QeA          0.011731
QgA          0.009931
urban        0.009730
wr_07        0.006551
wr_08        0.005207
Name: education, Length: 72, dtype: float64

In [11]:
# 연령대 칼럼과도 약한 선형관계를 보임
data[data.education!=0].groupby(['age_group'])['education'].mean()

age_group
+70s    3.016722
10s     1.780387
20s     2.863169
30s     3.000000
40s     3.004825
50s     2.978348
60s     3.022822
Name: education, dtype: float64

연령대 , 결혼 여부에 따라 그룹화 시킨 각 그룹의 교육 수준 비율을 참조해 난수를 생성해 처리

In [12]:
# 참조 테이블 생성
fill_table = data[(data.married!=0)&(data.education!=0)].groupby(['age_group','married'])['education'].value_counts(normalize=True).unstack() 
fill_table

education                 1         2         3         4
age_group married                                        
+70s      1        0.111111  0.333333  0.222222  0.333333
          2        0.028249  0.214689  0.322034  0.435028
          3        0.038835  0.407767  0.271845  0.281553
10s       1        0.323903  0.584934  0.079847  0.011317
          2        0.161290  0.688172  0.129032  0.021505
          3        0.322581  0.451613  0.161290  0.064516
20s       1        0.014825  0.297409  0.503444  0.184323
          2        0.013881  0.269621  0.464495  0.252002
          3        0.033708  0.446629  0.384831  0.134831
30s       1        0.024077  0.261867  0.445081  0.268975
          2        0.012370  0.212271  0.450272  0.325087
          3        0.029094  0.322527  0.413134  0.235245
40s       1        0.022963  0.274074  0.436296  0.266667
          2        0.010101  0.232026  0.422163  0.335710
          3        0.020243  0.316464  0.427800  0.235493
50s       1        0.020243  0.291498  0.445344  0.242915
          2        0.012055  0.256813  0.393082  0.338050
          3        0.028986  0.330616  0.379529  0.260870
60s       1        0.014706  0.161765  0.419118  0.404412
          2        0.010405  0.300578  0.351445  0.337572
          3        0.015909  0.313636  0.350000  0.320455

결측 데이터에 대하여 연령대 , 결혼 여부 정보를 참조하여 참조 테이블 값을 확률로한 난수로 채움

In [13]:
def fill_education_NA(row):
    if not row.married:
        total_prob = list(data.education[data.education!=0].value_counts(normalize=True).values) #특정 그룹이 아닌 전체에서 비율
        impute_value = np.random.choice([1,2,3,4],p=total_prob)
    else:
        impute_value = np.random.choice([1,2,3,4],p=list(fill_table.loc[(row.age_group,row.married)].values))
        
    row.education = impute_value
    return row

data[data.education==0] = data[data.education==0].apply(fill_education_NA,axis=1) 

In [14]:
data.education.value_counts()

2    21279
3    19283
4     9945
1     6405
Name: education, dtype: int64

### Urban

In [15]:
data.urban.unique() # NA 는 없지만 0 (무응답 존재)

array([1, 3, 2, 0], dtype=int64)

In [16]:
data.urban.value_counts() # 416개 무응답 데이터 

2    23084
3    22206
1    11206
0      416
Name: urban, dtype: int64

In [17]:
data.dropna().corr(method='spearman')['urban'].abs().sort_values(ascending=False) #engnat 컬럼과 약한 상관관계를 보임

urban         1.000000
engnat        0.153795
wr_01         0.082653
familysize    0.068431
wr_11         0.067428
                ...   
QaE           0.004758
tp10          0.003684
QfA           0.001968
QrE           0.001877
QlA           0.000181
Name: urban, Length: 72, dtype: float64

In [18]:
data[data.urban!=0].groupby(['race'])['urban'].value_counts(normalize=True).unstack() #race 칼럼의 정보에 따라 분포가 다름

urban,1,2,3
race,,,
Arab,0.087963,0.324074,0.587963
Asian,0.116497,0.317018,0.566485
Black,0.094065,0.357223,0.548712
Indigenous Australian,0.359375,0.453125,0.187500
Native American,0.324242,0.298485,0.377273
Other,0.122426,0.371771,0.505803
White,0.232556,0.439804,0.327641


engnat , race 컬럼 데이터로 그룹화 시킨 각 그룹의 urban 최빈값을 참조해 결측값 처리

In [19]:
# 참조 테이블 생성
fill_table = data[(data.engnat!=0)&(data.urban!=0)].groupby(['engnat','race'])['urban'].apply(lambda x : x.value_counts().index[0])
fill_table

engnat  race                 
1       Arab                     2
        Asian                    3
        Black                    3
        Indigenous Australian    2
        Native American          3
        Other                    3
        White                    2
2       Arab                     3
        Asian                    3
        Black                    3
        Indigenous Australian    1
        Native American          3
        Other                    3
        White                    3
Name: urban, dtype: int64

결측 데이터에 대하여 engnat , race 정보를 참조하여 참조 테이블 값으로 업데이트

In [20]:
def fill_urban_NA(row):
    if not row.engnat :
        total_prob = list(data.urban[data.urban!=0].value_counts(normalize=True).values) #특정 그룹이 아닌 전체에서 비율
        impute_value = np.random.choice(range(1,4),p=total_prob)
    else: 
        impute_value = fill_table[(row.engnat,row.race)]
        
    row['urban'] = impute_value
    return row

data[data.urban==0] = data[data.urban==0].apply(fill_urban_NA,axis=1)

In [21]:
data.urban.value_counts()

2    23308
3    22396
1    11208
Name: urban, dtype: int64

### Engnat

In [22]:
data.engnat.unique() # NA는 없지만 0(무응답) 존재

array([1, 2, 0], dtype=int64)

In [23]:
data.engnat.value_counts() # 94개 무응답 데이터

1    41295
2    15523
0       94
Name: engnat, dtype: int64

engnat 가 무응답인 데이터에 대해 다른 모든 (결측이 없는) 개인정보 관련 칼럼이 동일하거나 비슷한 그룹을 찾아 과반수 이상 값으로 업데이트

In [24]:
def fill_engnat_NA(row):  
    try:
        impute_value = subset[tuple([row[col] for col in cols])]
    except KeyError:
        total_prob = list(data[data.engnat!=0].value_counts(normalize=True).values)
        impute_value = np.random.choice([1,2],p=total_prob)
    row.engnat = impute_value
    return row

cols = ['age_group','gender','race','religion']
subset = data[data.engnat!=0].groupby(cols)['engnat'].apply(lambda x: x.value_counts().index[0])
data[data.engnat==0] = data[data.engnat==0].apply(fill_engnat_NA,axis=1) 

In [25]:
data.engnat.value_counts()

1    41372
2    15540
Name: engnat, dtype: int64

### Married

In [26]:
data.married.unique() # NA 는 없지만 0(무응답은 존재)

array([3, 1, 2, 0], dtype=int64)

In [27]:
data.married.value_counts() # 121개 무응답 데이터

1    39476
2    12528
3     4787
0      121
Name: married, dtype: int64

스피어만 상관계수 기준 education 칼럼과 약한 상관관계를 보임 

In [28]:
data.corr(method='spearman')['married'].abs().sort_values(ascending=False)

married      1.000000
education    0.291238
QqA          0.191236
QkA          0.188572
tp02         0.182331
               ...   
tp05         0.026242
wr_08        0.015263
wf_02        0.014967
hand         0.014185
QeA          0.008795
Name: married, Length: 72, dtype: float64

age_group 데이터에 따른 그룹핑 결과 연령대에 따라서도 몇몇 그룹에서 분포가 다른것을 확인

In [29]:
data[data.married!=0].groupby(['age_group'])['married'].value_counts().unstack()

married,1,2,3
age_group,,,
+70s,18,182,105
10s,17639,93,31
20s,15386,1895,358
30s,4420,4099,1225
40s,1366,3420,1503
50s,506,1952,1118
60s,141,887,447


age_group , education  칼럼의 정보로 그룹화 후 각 그룹의 최빈값을 결측 대체값으로 활용

In [30]:
# 최빈값 참조 테이블 생성
fill_table = data[(data.married!=0)&(data.education!=0)].groupby(['age_group','education'])['married'].apply(lambda x : x.value_counts().index[0])
fill_table

age_group  education
+70s       1            2
           2            3
           3            2
           4            2
10s        1            1
           2            1
           3            1
           4            1
20s        1            1
           2            1
           3            1
           4            1
30s        1            1
           2            1
           3            1
           4            2
40s        1            2
           2            2
           3            2
           4            2
50s        1            3
           2            2
           3            2
           4            2
60s        1            2
           2            2
           3            2
           4            2
Name: married, dtype: int64

결측 데이터에 대하여 age_group , education 정보를 참조하여 참조 테이블 값으로 업데이트

In [31]:
def fill_married_NA(row):
    if row.education == 0 :
        if row.age_group =='10s':
            fill_value = fill_table[(row.age_group,2)]
        else:
            fill_value = fill_table[(row.age_group,3)]
    else:
        fill_value = fill_table[(row.age_group,row.education)]
    
    row.married=fill_value
    return row

data[data.married==0] = data[data.married==0].apply(fill_married_NA , axis=1)

In [32]:
data.married.value_counts()

1    39563
2    12561
3     4788
Name: married, dtype: int64

### Hand

In [33]:
data.hand.unique() # NA는 없지만 0(무응답) 데이터 존재

array([1, 2, 3, 0], dtype=int64)

In [34]:
data.hand.value_counts() # 201개 무응답 데이터

1    48874
2     5847
3     1990
0      201
Name: hand, dtype: int64

결측을 제외한 hand 데이터의 전체 데이터 기준 비율을 활용하여 난수를 생성하여 결측값 대체

In [35]:
def fill_hand_NA(row):
    row.hand = np.random.choice([1,2,3],p=total_prob)
    return row

total_prob = list(data.hand[data.hand!=0].value_counts(normalize=True).values)
data[data.hand==0] = data[data.hand==0].apply(fill_hand_NA,axis=1) 

In [36]:
data.hand.value_counts()

1    49039
2     5870
3     2003
Name: hand, dtype: int64

# save file

In [37]:
train_nafilled = data.iloc[:split_point].join(train.voted)
test_nafilled = data.iloc[split_point:]
train_nafilled.shape,test_nafilled.shape

((45529, 77), (11383, 76))

In [38]:
train_nafilled.to_csv(filepath_home+'train_na_filled.csv',index=False)
test_nafilled.to_csv(filepath_home+'test_na_filled.csv',index=False)